In [2]:
import pandas as pd
import numpy as np

### Formula
<div class=hidden>
### Estimated rating for svd++  
$$\hat{r}_{ui}=\mu+b_i+b_u+q_i^T\left(p_u+|N(u)|^{\frac{1}{2}}\sum_{j\in N(u)}y_j\right)$$ 

### Loss function (MSE):
$$ \underset{b_*,\  q_*,\  p_*,\  y_*}{min}\sum_{(u,i)\in \kappa}\left[ r_{ui}-\mu - b_u - b_i -q_i^T\left(p_u+|N(u)|^{\frac{1}{2}}\sum_{j\in N(u)}y_j\right)\right]^2 + \lambda\left[b_u^2+b_i^2+\|q_i\|^2+\|p_u\|^2+\sum_{j\in N(u)}\|y_j\|^2\right]$$

Variable/Knowns:
* $r_{u,i}$: user u's rating on item i
* $u$: user
* $i$: item
* $\kappa$: known ratings

Hyperparameter:
* $k$: number of hidden features
* $\lambda$: regularization
* $\alpha$: learning rate


Parameters to learn:
* $b_i$ : [n_item, k]
* $b_u$ : [n_user, k]
* $q_i$ : []
* $p_u$ : []
* $y_i$ : []


Constants:
</div>

### Loading and split data, create our own MSE:

In [3]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv(
    './ml-100k/u.data', sep='\t', names=names)

n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]

# Create r_{ui}, our ratings matrix
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]

# Split into training and test sets. 
# Remove 10 ratings for each user 
# and assign them to the test set
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                        size=10, 
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test

train, test = train_test_split(ratings)
indicating_mat = np.vectorize(lambda x: 0 if x==0 else 1)(train)
mask = indicating_mat == 1

from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [5]:
test.nonzero()[0].shape

(9430,)

In [6]:
train.nonzero()[0].shape

(90570,)

In [7]:
ratings.nonzero()[0].shape

(100000,)

In [3]:
n_user, n_item = train.shape

# Hyperparameter:
k = 40
steps = 10
lrate = 0.001
_lambda = 0.001

# Parameters:
b_u = np.zeros(n_user)
b_i = np.zeros(n_item)

#q_i
q_i = np.random.normal(scale=1./k, size=(n_item, k))
#P_u
p_u = np.random.normal(scale=1./k, size=(n_user, k))
#y_i
y_j = np.random.normal(scale=1./k, size=(n_item, k))

global_bias = np.mean(train[np.where(train != 0)])

non_zeros = train.nonzero()

# This is equivalent of taking the length and doing the square root.
# N = np.power(indicating_mat.sum(1), -0.5)
N = 1./np.linalg.norm(indicating_mat, axis = 1)

def predict_one(u, i, b_i, b_u, q_i, q_u, y_j):
    return global_bias + b_i[i] + b_u[u] + \
    q_i[i,:].T.dot(p_u[u, :] + N[u] * y_j[mask[u,:], :].sum(axis=0))

def svdpp_step():
    rows = np.random.permutation(len(non_zeros[0]))
    for i in rows:
        user = non_zeros[0][i]
        item = non_zeros[1][i]
        pred = predict_one(user, item, b_i, b_u, q_i, p_u, y_j)
        ## Watch out to turn learning rate separately, this needs to be calculate separately
        error = train[user][item] - pred
        
        b_u[user] += lrate*(error - _lambda * b_u[user])
        b_i[item] += lrate*(error - _lambda * b_i[item])
        
        ## Update for q_i (item vector)
        q_i[item, :] += lrate * (error * (p_u[user,:] + N[user]* y_j[mask[user, :], :].sum(axis=0)) - _lambda * q_i[item, :])
        
        # Update for p_u (user vector)
        p_u[user, :] += lrate * (error * q_i[item, :] - _lambda * p_u[user,:])
        
        # Update for each y_j 
        temp = error * N[user] * q_i[item, :]
        j = indicating_mat[user,:].nonzero()
#         for j in indicating_mat[user,:].nonzero()[0]:
        y_j[j,:] += lrate * (temp - _lambda * y_j[j,:])

In [ ]:
data = []
def predict():
    predictions = np.zeros([n_user, n_item])
    for user in range(n_users):
        for item in range(n_items):
            predictions[user, item] = predict_one(user, item, b_i, \
                        b_u, q_i, p_u, y_j)
    data.append([get_mse(predictions, train),get_mse(predictions, test)])

In [8]:
for i in range(400):
    print(i)
    svdpp_step()
    if i%10 == 0:
        predict()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
dtf = pd.DataFrame(data)
dtf.to_csv("svd++Result.csv")

In [10]:
# get_mse(predictions, test)

0.932733067743351

In [11]:
# get_mse(predictions, train)

0.8126666901524443